In [29]:
from tkinter import *
import tkinter.messagebox
import math
import time

In [62]:
class Cell:  
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def right(self):
        return Cell(self.x,self.y+1)
    def left(self):
        return Cell(self.x,self.y-1)
    def up(self):
        return Cell(self.x-1,self.y)
    def down(self):
        return Cell(self.x+1,self.y)
    def distance(self,other):
        return math.sqrt(((other.x-self.x)**2)+((other.y-self.y)**2))
    def isEqual(self,other):
        return self.x==other.x and self.y==other.y
    
class Model:
    def __init__(self):
        self.grid_width = 10
        self.grid_height = 10
        self.empty = "-"
        self.ped = "P"
        self.obs = "O"
        self.tar = "T"
        self.obstacles = [Cell(4,5)]
        self.pedestrians = [Cell(0,0),Cell(2,3)]
        self.finishedPedestrians = [False for i in range(len(self.pedestrians))]
        self.targets = [Cell(6,7),Cell(8,2)]
        self.grid = self.createGrid()
        self.placeStates()
        #self.root = tk.Tk()
        #self.root.title("Cellular Automaton")
        #self.root.resizable(width=False,height=False)
        #self.w = tk.Canvas(self.root, width=750, height=750)
        #self.w.pack()
        #self.drawGrid2()
        ##self.drawGrid()
        


    def createGrid(self):
        return [ [self.empty]*self.grid_width for i in range(self.grid_height)]
    def placeStates(self):
        for p in self.pedestrians:
            self.grid[p.x][p.y] = self.ped
        for p in self.obstacles:
            self.grid[p.x][p.y] = self.obs
        for p in self.targets:
            self.grid[p.x][p.y] = self.tar
    
    def drawGrid(self):
        for i in range(self.grid_height):
            for j in range(self.grid_width):
                print(self.grid[i][j],end=" ")
            print()
        print("=====")
        
        
        
    def allFinished(self):
        for p in self.finishedPedestrians:
            if not p:
                return False
        return True
    def simulate(self):
        while not self.allFinished():
            self.simulateOneStep()
    def update(self,p,shortestCell,i,finished):
        oldPositionOnWindow = 2*(p.y+p.x*self.grid_width)+1
        newPositionOnWindow = 2*(shortestCell.y+shortestCell.x*self.grid_width)+1
#         self.w.itemconfig(oldPositionOnWindow, fill="white")
#         self.w.itemconfig(oldPositionOnWindow+1,text="E")
#         self.w.itemconfig(newPositionOnWindow, fill="red")
#         self.w.itemconfig(newPositionOnWindow+1,text="P") 
        
        
        self.grid[p.x][p.y] = self.empty
        self.grid[shortestCell.x][shortestCell.y] = self.ped
        
        self.pedestrians[i].x = shortestCell.x
        self.pedestrians[i].y = shortestCell.y
        
        if finished:
            self.finishedPedestrians[i] = True

    def simulateOneStep(self):
        for i in range(len(self.pedestrians)):
            if not self.finishedPedestrians[i]:
                p = self.pedestrians[i]
                shortestCell,finished = self.findShortestMove(p)
                self.update(p,shortestCell,i,finished)
                ##self.drawGrid()
                #self.w.update()
                #self.drawGrid2()
                #self.w.update()
                #if not self.allFinished():
                #    self.w.after(5000, self.simulateOneStep)
            
    def isValid(self,p):
        return p.x>=0 and p.x<self.grid_height and p.y>=0 and p.y<self.grid_width and self.grid[p.x][p.y] == self.empty
    def findShortestMove(self,p):
        currentDistance =p.distance(self.targets[0])
        shortestCell = p
        for target in self.targets:
            if self.isValid(p.left()) and p.left().distance(target) < currentDistance:
                currentDistance = p.left().distance(target)
                shortestCell = p.left()
            if self.isValid(p.right()) and p.right().distance(target) < currentDistance:
                currentDistance = p.right().distance(target)
                shortestCell = p.right()
            if self.isValid(p.up()) and p.up().distance(target) < currentDistance:
                currentDistance = p.up().distance(target)
                shortestCell = p.up()
            if self.isValid(p.down()) and p.down().distance(target) < currentDistance:
                currentDistance = p.down().distance(target)
                shortestCell = p.down()
        if currentDistance == 1:
            finished = True
        else:
            finished = False
        return shortestCell,finished
    

        
        

In [63]:
def display(simulation):
    for i in range(simulation.grid_height):
        for j in range(simulation.grid_width):
            if simulation.grid[i][j] == 'P':
                canvas.create_rectangle(50+30*j, 25+30*i, 80+30*j, 55+30*i, fill="red",outline="black")
                canvas.create_text(50+30*j+15,25+30*i+15,text ="P")
            elif simulation.grid[i][j] == 'O':
                canvas.create_rectangle(50+30*j, 25+30*i, 80+30*j, 55+30*i, fill="purple",outline="black")
                canvas.create_text(50+30*j+15,25+30*i+15,text ="O")
            elif simulation.grid[i][j] == 'T':
                canvas.create_rectangle(50+30*j, 25+30*i, 80+30*j, 55+30*i, fill="goldenrod",outline="black")
                canvas.create_text(50+30*j+15,25+30*i+15,text ="T")
            else:
                canvas.create_rectangle(50+30*j, 25+30*i, 80+30*j, 55+30*i, fill="white",outline="black")
                canvas.create_text(50+30*j+15,25+30*i+15,text =" ")

In [70]:
simulation = Model()
root = tkinter.Tk()
root.title("Cellular Automaton")
root.resizable(width=False,height=False)
canvas = tkinter.Canvas(root, width=400, height=400)
currentGrid = simulation.grid
display(simulation)

canvas.pack()
def nextStep():
    canvas.delete("all")
    simulation.simulateOneStep()
    currentGrid = simulation.grid
    display(simulation)
def emptyCell():
    cellToDelete = Cell(rowNum.get()-1,colNum.get()-1)
    for i in range(len(simulation.pedestrians)):
        if simulation.pedestrians[i].isEqual(cellToDelete):
            del simulation.pedestrians[i]
            del simulation.finishedPedestrians[i]
            break
    for i in range(len(simulation.obstacles)):
        if simulation.obstacles[i].isEqual(cellToDelete):
            del simulation.obstacles[i]
            break
    for i in range(len(simulation.targets)):
        if simulation.targets[i].isEqual(cellToDelete):
            del simulation.targets[i]
            break
    simulation.grid[cellToDelete.x][cellToDelete.y] = simulation.empty
    
    display(simulation)
def addPedestrian():
    cellToAdd = Cell(rowNum.get()-1,colNum.get()-1)
    simulation.pedestrians.append(cellToAdd)
    simulation.grid[cellToAdd.x][cellToAdd.y] = simulation.ped
    simulation.finishedPedestrians.append(False)
    display(simulation)
def addObstacle():
    cellToAdd = Cell(rowNum.get()-1,colNum.get()-1)
    simulation.obstacles.append(cellToAdd)
    simulation.grid[cellToAdd.x][cellToAdd.y] = simulation.obs
    display(simulation)
def addTarget():
    cellToAdd = Cell(rowNum.get()-1,colNum.get()-1)
    simulation.targets.append(cellToAdd)
    simulation.grid[cellToAdd.x][cellToAdd.y] = simulation.tar
    display(simulation)
    
    
clickButton = Button(root, text = "NEXT STEP", command = nextStep).pack()
rowNum = tkinter.IntVar()
colNum = tkinter.IntVar()
rowLabel = Label(root, text = 'Row Number [1-10]').pack()
rowEntry = Entry(root,textvariable = rowNum).pack()
colLabel = Label(root, text = 'Column Number [1-10]').pack()
colEntry=Entry(root, textvariable = colNum).pack()
addPedestrian = Button(root, text = "Add Pedestrian", command = addPedestrian).pack()
addObstacle = Button(root, text = "Add Obstacle", command = addObstacle).pack()
addTarget = Button(root, text = "Add Target", command = addTarget).pack()
emptyCell = Button(root, text = "Delete Cell", command = emptyCell).pack()
root.mainloop()